In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords
from sqlalchemy import create_engine
import re

# Tải Dữ liệu 
DB_CONFIG = {
    "dbname": "olist_db",
    "user": "postgres", 
    "password": "YOUR_DB_PASSWORD",
    "host": "localhost",
    "port": 5432,
}

conn_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
engine = create_engine(conn_string)

# Tải stopwords tiếng Bồ Đào Nha
nltk.download('stopwords')
pt_stopwords = stopwords.words('portuguese')

# Thêm các từ vô nghĩa thường gặp trong review TMĐT vào danh sách loại bỏ
custom_stops = ['produto', 'entrega', 'prazo', 'antes', 'chegou', 'recomendo', 'muito', 'bom', 'bem', 'tido', 'fazer', 'loja', 'vendedor']
pt_stopwords.extend(custom_stops)

# 1. TRUY VẤN DỮ LIỆU
# Chỉ lấy review 4-5 sao (Khách hài lòng) và có nội dung dài > 3 từ

df_topics = pd.read_sql_table('nlp_good_review', engine, schema='warehouse')
print(f"Số lượng review để phân tích: {len(df_topics):,}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinhq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Số lượng review để phân tích: 75,917


In [3]:
# 2. HÀM LÀM SẠCH TEXT
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # Bỏ dấu câu
    text = re.sub(r'\d+', '', text)     # Bỏ số
    return text

df_topics['clean_text'] = df_topics['review_comment_message'].apply(clean_text)

# 3. VECTOR HÓA (CountVectorizer)
# LDA cần tần suất từ (Bag of Words), không cần TF-IDF
print("Đang vector hóa văn bản...")
vectorizer = CountVectorizer(stop_words=pt_stopwords, max_features=2000, max_df=0.9, min_df=5)
doc_term_matrix = vectorizer.fit_transform(df_topics['clean_text'])

# 4. CHẠY MÔ HÌNH LDA
# Giả sử ta muốn tìm 5 chủ đề chính ẩn sau các lời khen
NUM_TOPICS = 10
print(f"Đang chạy LDA để tìm {NUM_TOPICS} chủ đề...")

lda = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42, n_jobs=-1)
lda.fit(doc_term_matrix)

# 5. HIỂN THỊ KẾT QUẢ
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nChủ đề {topic_idx + 1}:")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print("-" * 30)
print("CÁC CHỦ ĐỀ CHÍNH CỦA KHÁCH HÀNG HÀI LÒNG (HAPPY CHURNERS):")
display_topics(lda, vectorizer.get_feature_names_out(), 10)
print("-" * 30)

Đang vector hóa văn bản...
Đang chạy LDA để tìm 10 chủ đề...
------------------------------
CÁC CHỦ ĐỀ CHÍNH CỦA KHÁCH HÀNG HÀI LÒNG (HAPPY CHURNERS):

Chủ đề 1:
qualidade, ótima, adorei, satisfeita, lindo, data, super, bonito, material, linda

Chủ đề 2:
esperado, veio, comprei, recebi, pois, pena, maravilhoso, achei, obrigado, vale

Chủ đề 3:
perfeito, amei, estado, obrigada, tudo, pra, foto, demorou, correios, pouco

Chủ đề 4:
compra, satisfeito, site, certinho, tudo, compras, fiquei, comprar, eficiente, fiz

Chủ đề 5:
gostei, ok, otimo, tudo, preço, combinado, melhor, rapido, bonita, bastante

Chủ đề 6:
excelente, entregue, dentro, expectativas, atendimento, entregou, previsto, atendeu, estipulado, superou

Chủ đề 7:
rápida, super, rápido, ainda, veio, embalado, tempo, embalagem, porém, caixa

Chủ đề 8:
ótimo, conforme, parabéns, entregue, condições, anunciado, perfeitas, acordo, pedido, descrição

Chủ đề 9:
boa, sempre, produtos, qualidade, comprar, lannister, parabéns, nota, lojas